<a href="https://colab.research.google.com/github/Nitin-Dwivedi-7/COVID_19-Test-Prediction-Using-CT-scan-Images/blob/main/ResNet50_major_25GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import Keras packages
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import load_img
import random

import numpy as np
from keras import applications
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.utils import get_file

In [ ]:
import pickle 
file = "/content/drive/My Drive/data_5K.pkl"
f_o=open(file,"rb")
data=pickle.load(f_o)
f_o.close()
file= "/content/drive/My Drive/labels_5k.pkl"
f_o=open(file,"rb")
labels=pickle.load(f_o)
f_o.close()

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
img_width, img_height = 256, 256        # Resolution of inputs
#train_data_dir = "train"           # Folder of train samples
#validation_data_dir = "val" # Folder of validation samples44
#nb_train_samples = 10000                # Number of train samples
#nb_validation_samples = 9500            # Number of validation samples
batch_size = 64                        # Batch size
epochs = 20                # Maximum number of epochs

In [ ]:
# Load ResNet50 

model=applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))



In [ ]:
# Freeze first 15 layers
for layer in model.layers[:45]:
	layer.trainable = False
for layer in model.layers[45:]:
   layer.trainable = True

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x) # 4-way softmax classifier at the end


In [ ]:
#nb_train_samples = 2543                # Number of train samples
#nb_validation_samples = 2261            # Number of validation samples

# Initializing the CNN


In [ ]:
from keras.utils.np_utils import to_categorical   

categorical_labels = to_categorical(labels, num_classes=2)


from sklearn.model_selection import train_test_split

#X_train1, X_test0, ytrain1, ytest0 = train_test_split(data, categorical_labels, test_size=0.1,
                                                    #random_state=random.randint(0,100))
X_train1 = data
ytrain1 = categorical_labels

for index in range(10):
  classifier = Model(model.input,predictions)
  classifier.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-3, momentum=0.9), metrics=["accuracy"])


X_train, X_test1, ytrain, ytest1 = train_test_split(X_train1, ytrain1, test_size=0.1,
                                                    random_state=random.randint(0,100))
X_val, X_test, yval, ytest = train_test_split(X_test1, ytest1, test_size=0.5,
                                                    random_state=random.randint(0,100))
    


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
    
train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    
test_datagen = ImageDataGenerator(rescale=1./255)
    
val_datagen = ImageDataGenerator(rescale=1./255)
    
training_set = train_datagen.flow(
            X_train, ytrain,
            batch_size=64)
    
val_set = val_datagen.flow(
            X_val, yval,
            batch_size=64)
    
test_set = test_datagen.flow(
            X_test, ytest,
            batch_size=64)
    
X, y = test_set.next()
    
classifier.fit_generator(
            training_set,
            steps_per_epoch=20,
            epochs=epochs,
            validation_data=val_set,
            validation_steps=100)
    
    #w_file = 'Wilson_vgg16_model_weights_k10.h5'
    #classifier.save_weights(w_file)
    
    
    

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
20/20 [==============================] - 19s 956ms/step - loss: 1.1355 - accuracy: 0.6875 - val_loss: 0.8585 - val_accuracy: 0.5111
Epoch 2/20
20/20 [==============================] - 17s 849ms/step - loss: 0.6780 - accuracy: 0.8578
Epoch 3/20
20/20 [==============================] - 17s 850ms/step - loss: 0.2724 - accuracy: 0.9148
Epoch 4/20
20/20 [==============================] - 17s 853ms/step - loss: 0.1411 - accuracy: 0.9609
Epoch 5/20
20/20 [==============================] - 17s 844ms/step - loss: 0.0775 - accuracy: 0.9740
Epoch 6/20
20/20 [==============================] - 17s 843ms/step - loss: 0.0549 - accuracy: 0.9867
Epoch 7/20
20/20 [==============================] - 17s 848ms/step - loss: 0.0553 - accuracy: 0.9898
Epoch 8/20
20/20 [==============================] - 17s 850ms/step - loss: 0.0524 - accuracy: 0.9859
Epoch 9/20
20/20 [==============================] - 17s 849ms/step - loss:

In [ ]:
arr = classifier.evaluate(X,y)

2/2 [==============================] - 0s 42ms/step - loss: 0.2191 - accuracy: 0.9062


In [ ]:
print(arr)

[0.21905159950256348, 0.90625]


In [ ]:


arr = classifier.predict(X)
arr = np.argmax(arr, axis=1)

print(arr)
print(y)


[0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1
 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 1]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [ ]:
#print(X.shape)

In [ ]:
#print(X[::-1])

In [ ]:
    #array = np.reshape(X,(64,256,256,3)) 


In [ ]:
#from PIL import Image
"""import numpy as np

img = Image.fromarray(array, 'RGB')
img.save('test.png')
img.show()"""

"import numpy as np\n\nimg = Image.fromarray(array, 'RGB')\nimg.save('test.png')\nimg.show()"

In [ ]:
"""x=[]

image1=load_img("/content/real_CT3_A.png",  
    target_size= (256,256))
image1=np.array(image1)

x.append(image1)
x = np.array(x)
arr = classifier.predict(x)
#print(arr)"""

[[1.6399764e-16 1.0000000e+00]]


In [ ]:
#x.shape

(1, 256, 256, 3)

In [ ]:
#array = np.reshape(x,(256,256,3)) 

In [ ]:
"""from PIL import Image
import numpy as np

img = Image.fromarray(array, 'RGB')
img.save('tes111100000t.png')
img.show()
"""

In [ ]:
X[0].shape

(256, 256, 3)

In [ ]:
X[0]

array([[[0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        ...,
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706]],

       [[0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        ...,
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706]],

       [[0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        ...,
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706]],

       ...,

       [[0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        [0.1764706, 0.1764706, 0.1764706],
        ...,
        [0.1764706, 0.1764

In [ ]:
x=[]
image1=load_img("/content/real_CT3_A.png",  
    target_size= (256,256))
image1=np.array(image1)

x.append(image1)
x = np.array(x)

In [ ]:
x.shape

(1, 256, 256, 3)

In [ ]:
x[0]

array([[[ 2,  2,  2],
        [ 5,  5,  5],
        [ 4,  4,  4],
        ...,
        [ 4,  4,  4],
        [ 6,  6,  6],
        [10, 10, 10]],

       [[ 1,  1,  1],
        [ 4,  4,  4],
        [ 2,  2,  2],
        ...,
        [ 4,  4,  4],
        [ 5,  5,  5],
        [ 6,  6,  6]],

       [[ 0,  0,  0],
        [ 3,  3,  3],
        [ 2,  2,  2],
        ...,
        [ 4,  4,  4],
        [ 3,  3,  3],
        [ 4,  4,  4]],

       ...,

       [[14, 14, 14],
        [15, 15, 15],
        [16, 16, 16],
        ...,
        [18, 18, 18],
        [20, 20, 20],
        [25, 25, 25]],

       [[14, 14, 14],
        [18, 18, 18],
        [18, 18, 18],
        ...,
        [17, 17, 17],
        [21, 21, 21],
        [26, 26, 26]],

       [[17, 17, 17],
        [22, 22, 22],
        [21, 21, 21],
        ...,
        [15, 15, 15],
        [19, 19, 19],
        [26, 26, 26]]], dtype=uint8)

In [ ]:
x.rescale=1./255

AttributeError: ignored

AttributeError: ignored